# 版本变更说明
**enbedding 段的 input_file 是《way1\1_merged table v1.xlsx》**，它有如下特点：  

- 来路明确，是我用《way1\1_merge_table.ipynb》脚本合并 WTOS 和 Data power 文件而成

**enbedding 段增加了矫正乱码的代码**，为方便检查，我把原始letter列保存在 orginal_letter 列

In [8]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

# embedding

In [9]:
input_file = r"V:\20240920\way1\1_merged table v1.xlsx"

csv_output_file = r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv"
npy_output_file = r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"

print("Loading data...")

df = pd.read_excel(input_file, sheet_name="sheet1")
df['orginal_letter'] = df['letter']

## replace the specific mistake characters
#详见《 letter列文本中的乱码及处理方法.xlsx》 
replacements = {
            '‚Äô': "'",
            '‚äô': "'",
            '√©': " ",
            'Äì¬†': " ",
            '‚Äú': "\"",
            '‚Äù': "\'",
            '‚Äî': "--",
            '‚Ä¶': " ",
            '√≥': "o"
        }
for old, new in replacements.items():
    df['letter'] = df['letter'].str.replace(old, new, regex=False)


# Remove None values and show statistics
print("\nData statistics:")
print(f"Total number of texts: {len(df)}")
print(f"Number of None values: {df['letter'].isnull().sum()}")
   
# Remove rows with None values
# 必须删除 'letter' 列为空的行，否则 SentenceTransformer 会报错
df_clean = df.dropna(subset=['letter'])
print(f"Number of valid texts after removing None: {len(df_clean)}")

# Creating embeddings
print("Creating embeddings...")
texts = df_clean['letter'].tolist()
# texts = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer(r"V:\huggingface\model\sentence-transformers/all-mpnet-base-v2") # off-line mode
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2') # on-line mode

embeddings = model.encode(texts, max_length=2048)

# Saving embeddings  
print("\nSaving embeddings...")
# np_embeddings = embeddings.numpy()
np.save(npy_output_file, embeddings)

# compare length of embedding and data file 
print("\n --- compare length of embedding and data file ---")
print(f"Embeddings shape: {embeddings.shape}")
print(f"length of csv_output_file: {len(df_clean['letter'])}")

# Save clean data for reference
df_clean.to_csv(csv_output_file, index=False)

Loading data...

Data statistics:
Total number of texts: 8845
Number of None values: 3262
Number of valid texts after removing None: 5583
Creating embeddings...

Saving embeddings...

 --- compare length of embedding and data file ---
Embeddings shape: (5583, 768)
length of csv_output_file: 5583


# PCA

## explain variance percentage

In [10]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def components_for_variance(embeddings_path, n):
    embed = np.load(embeddings_path)
    scaler = StandardScaler()
    scaled_embeddings = scaler.fit_transform(embed)

    pca = PCA()
    pca.fit(scaled_embeddings)
    
    # Calculate cumulative variance ratio
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    
    num_components = np.argmax(cumulative_variance >= n) + 1  
    
    print(f"Need {num_components} components to explain {n * 100}% variance.")
    return num_components


embeddings_path = r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"

components_for_variance(embeddings_path, 0.7)
components_for_variance(embeddings_path, 0.8)
components_for_variance(embeddings_path, 0.9)


Need 35 components to explain 70.0% variance.
Need 65 components to explain 80.0% variance.
Need 138 components to explain 90.0% variance.


138

## Themes analysis

In [12]:
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# select explained variance ratio
explained_variance_ratio = 0.7
number_components = components_for_variance(embeddings_path, explained_variance_ratio)
print(f"\n --- need {number_components} components for explained variance ratio {explained_variance_ratio} ---\n")

def analyze_pca_themes(embeddings_path=r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy", 
                       data_path=r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv",
                       n_components = 34,   
                       n_top_texts=5):
    # Load data
    print("Loading data...")
    embeddings = np.load(embeddings_path)
    df = pd.read_csv(data_path)

    # compare length of embedding and data file 
    print("\n --- compare length of embedding and data file ---")
    print(f"Embeddings shape: {embeddings.shape}")
    print(f"length of data file: {len(df['letter'])}")
    
    # Check data length consistency
    if embeddings.shape[0] != len(df):
        raise ValueError("The number of embeddings does not match the number of texts in the data file.")
  
    # Standardize embeddings
    print("\n--- Standardizing embeddings...")
    scaler = StandardScaler()
    scaled_embeddings = scaler.fit_transform(embeddings)
   
    # Apply PCA
    print(f"\n--- Applying PCA with {n_components} components...")
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(scaled_embeddings)
   
    print(f"\n --- Total explained variance: {sum(pca.explained_variance_ratio_):.4f}")
    
    # Print explained variance ratio per component
    print("\n--- Explained Variance Ratio per Component ---")
    for i, ratio in enumerate(pca.explained_variance_ratio_):
        print(f"Component {i+1}: {ratio:.4f}")
   
    # Analyze themes
    print("\n\n --- Analyzing themes in each component...")
    themes = {}
    for i in range(n_components):
        # Get component scores
        scores = pca_result[:, i]
        
        # Get top and bottom texts
        top_indices = np.argsort(scores)[-n_top_texts:]
        bottom_indices = np.argsort(scores)[:n_top_texts]
        
        # Add statistics
        themes[f'Component_{i+1}'] = {
            'positive': [(df['letter'].iloc[idx], scores[idx]) 
                         for idx in reversed(top_indices)],
            'negative': [(df['letter'].iloc[idx], scores[idx]) 
                         for idx in bottom_indices],
            'score_stats': {
                'mean': np.mean(scores),
                'std': np.std(scores),
                'min': np.min(scores),
                'max': np.max(scores)
            }
        }
   
    return pca_result, themes

def save_themes_to_excel(themes, output_path):
    """
    Save the thematic analysis results to an Excel file.
    """
    # Step 1: Check if the file exists and delete it
    if os.path.exists(output_path):
        os.remove(output_path)
        print(f"old File '{output_path}' has been deleted.")
    else:
        print(f"File '{output_path}' does not exist. Creating a new file.")
    
    # Create a Pandas Excel writer using XlsxWriter as the engine
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for component, theme_data in themes.items():
            # Create a DataFrame for positive examples
            positive_df = pd.DataFrame(theme_data['positive'], columns=['Text', 'Score'])
            positive_df.to_excel(writer, sheet_name=f"{component}_Positive", index=False)
            
            # Create a DataFrame for negative examples
            negative_df = pd.DataFrame(theme_data['negative'], columns=['Text', 'Score'])
            negative_df.to_excel(writer, sheet_name=f"{component}_Negative", index=False)
            
            # Add statistics to the Excel file
            stats_df = pd.DataFrame([theme_data['score_stats']])
            stats_df.to_excel(writer, sheet_name=f"{component}_Stats", index=False)

if __name__ == "__main__":
    # Apply PCA and analyze themes
    pca_result, themes = analyze_pca_themes(n_components = number_components)
    
    # Save PCA results
    np.save(r"V:\20240920\theme_analysis_act3301\pca_results.npy", pca_result)
    
    # Save themes to Excel file
    save_themes_to_excel(themes, r"V:\20240920\theme_analysis_act3301\thematic_analysis_results.xlsx")

Need 35 components to explain 70.0% variance.

 --- need 35 components for explained variance ratio 0.7 ---

Loading data...

 --- compare length of embedding and data file ---
Embeddings shape: (5583, 768)
length of data file: 5583

--- Standardizing embeddings...

--- Applying PCA with 35 components...

 --- Total explained variance: 0.7012

--- Explained Variance Ratio per Component ---
Component 1: 0.1087
Component 2: 0.0813
Component 3: 0.0642
Component 4: 0.0500
Component 5: 0.0372
Component 6: 0.0288
Component 7: 0.0268
Component 8: 0.0235
Component 9: 0.0224
Component 10: 0.0198
Component 11: 0.0191
Component 12: 0.0167
Component 13: 0.0153
Component 14: 0.0151
Component 15: 0.0134
Component 16: 0.0129
Component 17: 0.0115
Component 18: 0.0113
Component 19: 0.0101
Component 20: 0.0100
Component 21: 0.0093
Component 22: 0.0092
Component 23: 0.0085
Component 24: 0.0081
Component 25: 0.0078
Component 26: 0.0075
Component 27: 0.0071
Component 28: 0.0068
Component 29: 0.0065
Compone

# Varimax Rotation

In [13]:
from factor_analyzer.rotator import Rotator

# load data
embeddings_path=r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"
embeddings = np.load(embeddings_path)

# Standardize embeddings
print("Standardizing embeddings...")
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)

# 70% varianc explanation.
pca = PCA()
pca_result = pca.fit_transform(scaled_embeddings)
cumulative_variance = pca.explained_variance_ratio_.cumsum()
threshold = explained_variance_ratio         # keep same as themes analysis 
n_components = np.argmax(cumulative_variance >= threshold) + 1
pca = PCA(n_components = n_components)
pca.fit(scaled_embeddings)

# transposes the matrix so that each row corresponds to a feature and each column corresponds to a component.
loadings = pca.components_.T  
print("\n---Loadings:\n", loadings)
print(loadings.shape)
print(loadings.T.shape)

rotator = Rotator(method='varimax')
rotated_loadings = rotator.fit_transform(loadings)

print("\n---Rotated Loadings:\n", rotated_loadings)
print(f"Rotated Loadings's shape: {rotated_loadings.shape}")
print(pca.components_.shape)
print(pca.components_.T.shape)

Standardizing embeddings...

---Loadings:
 [[-0.05831511  0.01672853 -0.02240267 ... -0.01173237  0.0099031
   0.0402006 ]
 [-0.03672548  0.02988562  0.04441902 ... -0.00957623  0.01379264
   0.03293107]
 [ 0.03683105 -0.03172079 -0.00223721 ...  0.00243738  0.04210471
  -0.02097005]
 ...
 [-0.05399645 -0.04809344 -0.02057458 ...  0.01916452  0.06832749
  -0.02811958]
 [-0.0551717  -0.03301169  0.05814507 ...  0.00180005  0.0167277
   0.04576058]
 [-0.01073436 -0.01206578 -0.02260295 ... -0.03737618  0.03881303
   0.01323039]]
(768, 35)
(35, 768)

---Rotated Loadings:
 [[-0.05929761  0.03455264 -0.02050486 ... -0.03448478  0.01289598
   0.04559542]
 [ 0.0179012  -0.02260267  0.00486097 ... -0.012991    0.03041592
  -0.00812172]
 [-0.01424091  0.02183586  0.00840591 ...  0.02779863  0.00460952
  -0.02039335]
 ...
 [-0.0695141  -0.01609712 -0.03669699 ... -0.06024928  0.00341119
  -0.04632895]
 [-0.04060106 -0.05536942  0.00582074 ...  0.00981039 -0.027425
   0.00832968]
 [-0.01053765 -0

## get the new score after varimax rotation

In [14]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from factor_analyzer.rotator import Rotator

# Step 1: Load the Data
embeddings_path = r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"
data_path = r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv"

embeddings = np.load(embeddings_path)
df = pd.read_csv(data_path)

# Step 2: Standardize the Embeddings
print("Standardizing embeddings...")
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(embeddings)

# Step 3: Perform PCA and Varimax Rotation
# Perform PCA to determine the number of components explaining 90% variance
pca = PCA()
pca_result = pca.fit_transform(scaled_embeddings)
cumulative_variance = pca.explained_variance_ratio_.cumsum()
threshold = explained_variance_ratio         # keep same as themes analysis
n_components = np.argmax(cumulative_variance >= threshold) + 1

# Fit PCA with the selected number of components
pca = PCA(n_components=n_components)
pca.fit(scaled_embeddings)

# Extract PCA loadings and transpose them
loadings = pca.components_.T

# Apply Varimax rotation
rotator = Rotator(method='varimax')
rotated_loadings = rotator.fit_transform(loadings)

# Step 4: Project Data onto Rotated Components
# Calculate the new scores by multiplying the standardized embeddings with the rotated loadings
rotated_scores = np.dot(scaled_embeddings, rotated_loadings)

# Step 5: Save the New Scores
# Save the rotated scores to a new file
output_path = r"V:\20240920\theme_analysis_act3301\rotated_scores.npy"
np.save(output_path, rotated_scores)

# Print the shape of the rotated scores
print(f"Rotated Scores' shape: {rotated_scores.shape}")

# Optional: Save the rotated scores with corresponding text data to a CSV file
rotated_scores_df = pd.DataFrame(rotated_scores, columns=[f"Rotated_Component_{i+1}" for i in range(n_components)])
rotated_scores_df["letter"] = df["letter"].values
rotated_scores_df.to_csv(r"V:\20240920\theme_analysis_act3301\rotated_scores_with_text.csv", index=False)

Standardizing embeddings...
Rotated Scores' shape: (5583, 35)


## get top_n positive and negative score letter of each Rotated Component 

In [15]:
import pandas as pd

# Define the number of top scores to extract
top_n = 5

# Extract Top `top_n` Positive and Negative Scores for Each Rotated Component
results = []
for component in rotated_scores_df.columns[:-1]:  # Exclude the "letter" column
    # Get the top_n positive scores
    top_positive = rotated_scores_df.nlargest(top_n, component)[["letter", component]]
    
    # Get the top_n negative scores
    top_negative = rotated_scores_df.nsmallest(top_n, component)[["letter", component]]
    
    # Store the results for positive scores
    for letter, score in top_positive.values:
        results.append({
            "Component": component,
            "Type": "Positive",
            "Letter": letter,
            "Score": score
        })
    
    # Store the results for negative scores
    for letter, score in top_negative.values:
        results.append({
            "Component": component,
            "Type": "Negative",
            "Letter": letter,
            "Score": score
        })

# Step 3: Save the Results to a CSV File
# Create a DataFrame to store the results
results_df = pd.DataFrame(results)

# Save the results to a CSV file
output_path = r"V:\20240920\theme_analysis_act3301\top_scores_per_rotated_component.csv"
results_df.to_csv(output_path, index=False)

# Print the results
print(results_df)

                Component      Type  \
0     Rotated_Component_1  Positive   
1     Rotated_Component_1  Positive   
2     Rotated_Component_1  Positive   
3     Rotated_Component_1  Positive   
4     Rotated_Component_1  Positive   
..                    ...       ...   
345  Rotated_Component_35  Negative   
346  Rotated_Component_35  Negative   
347  Rotated_Component_35  Negative   
348  Rotated_Component_35  Negative   
349  Rotated_Component_35  Negative   

                                                Letter      Score  
0    I do not support the reintroduction of the bil...  13.588921  
1    I don't believe bill S. 3301 should be reintro...  11.836024  
2    Please don't reintroduce the Women's Global Em...  10.828992  
3    I feel as though we should not re-introduce th...  10.804545  
4    Dear Senator Casey,\n  I am writing to urge yo...  10.313177  
..                                                 ...        ...  
345  Dear Senator, please continue to oppose this b... 

# test

In [6]:
input_file = r"V:\20240920\way1\1_merged table v1.xlsx"

csv_output_file = r"V:\20240920\theme_analysis_act3301\act3301_processed_data_clean.csv"
npy_output_file = r"V:\20240920\theme_analysis_act3301\text_embeddings_clean_lb2.npy"

print("Loading data...")

df = pd.read_excel(input_file, sheet_name="sheet1")
df['orginal_letter'] = df['letter']

## Task1: replace the specific mistake characters
#详见《 letter列文本中的乱码及处理方法.xlsx》 
replacements = {
            '‚Äô': "'",
            '‚äô': "'",
            '√©': " ",
            'Äì¬†': " ",
            '‚Äú': "\"",
            '‚Äù': "\'",
            '‚Äî': "--",
            '‚Ä¶': " ",
            '√≥': "o"
        }
for old, new in replacements.items():
    df['letter'] = df['letter'].str.replace(old, new, regex=False)
        

Loading data...
